In [8]:
# import packages

import requests
import zipfile
import arcpy
import pprint
import json
import os
import csv
from io import BytesIO
import shutil
import arcpy.mp as mp
import numpy as np

In [2]:
mn_gsc_api_url = "https://gisdata.mn.gov/api/3/action"

# MN geospatial commons api search datasets request for parcels

search_datasets_url = mn_gsc_api_url + "/package_search"
query = "Parcels, Compiled from Opt-In Open Data Counties, Minnesota"
request_url = search_datasets_url + f"?q={query}"
response = requests.get(request_url, verify=False) # verify=False disables SSL certification validation which makes it mad but i cant get it to work without disabling it
response_data = json.loads(response.text)
found_dataset_names = [dataset["title"] for dataset in response_data["result"]["results"]]
print(found_dataset_names)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['Parcels, Compiled from Opt-In Open Data Counties, Minnesota', 'Status of Free and Open Public Geospatial Data from Minnesota Counties', 'Tax Parcels, Saint Louis County, Minnesota', 'Parcel Data Geospatial Advisory Council (GAC) Validation Tool', 'Racial Covenants in Hennepin County', 'Tax Parcels - Points, Lines and Polygons, Dakota County, Minnesota', 'Public Parcels - Metro CTUs', 'Street Names in Minnesota', 'Minnesota Walk-In Access Sites', 'Metro Regional Parcel Dataset - (Updated Quarterly)']


In [3]:
# MN geospatial commons api get dataset request for parcels

first_dataset_result_id = response_data["result"]["results"][0]["id"] # grab dataset that is first in returned dataset results
get_dataset_url = mn_gsc_api_url + "/package_show"
id = first_dataset_result_id
request_url = get_dataset_url + f"?id={id}"
response = requests.get(request_url, verify=False) # again disabling ssl cert validation
response_data = json.loads(response.text)
resources = response_data["result"]["resources"]
raw_zip_dataset_file_download_url = [resource for resource in resources if resource["resource_type"] == "fgdb"][0]["url"]
print(raw_zip_dataset_file_download_url)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mngeo/plan_parcels_open/fgdb_plan_parcels_open.zip


In [4]:
# download parcels
response = requests.get(raw_zip_dataset_file_download_url)
# check response.status_code to see if download was successful, should return '200'
os.chdir(r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject")
if response.status_code == 200:
    # Open zipfile
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        # extract contents of zipfile
        z.extractall('parcels') # extracts contents into a folder named 'parcels'
        
        # assuming there's only one landuse file in the zip, get its name
        parcels_file = [name for name in z.namelist()]
        print(parcels_file)

['Parcels - Open Data Counties in Minnesota - Metadata Layer.lyr', 'Parcels - Open Data Counties in Minnesota.lyr', 'metadata/metadata.html', 'metadata/Open Parcel Compilation Metadata Layer.xml', 'metadata/Open Parcel Compilation Metadata Layer.html', 'metadata/metadata.xml', 'metadata/preview.jpg', 'metadata/Thumbs.db', 'plan_parcels_open.gdb/a0000000b.gdbtable.cdf', 'plan_parcels_open.gdb/a0000000c.gdbtable.FDO_OBJECTID_1.gdbtabidx', 'plan_parcels_open.gdb/a00000004.gdbtable', 'plan_parcels_open.gdb/a00000006.gdbindexes', 'plan_parcels_open.gdb/a00000007.gdbindexes', 'plan_parcels_open.gdb/a00000007.CatRelTypesByBackwardLabel.atx', 'plan_parcels_open.gdb/a00000004.CatItemsByType.atx', 'plan_parcels_open.gdb/a00000007.CatRelTypesByForwardLabel.atx', 'plan_parcels_open.gdb/a00000007.gdbtable', 'plan_parcels_open.gdb/a00000004.gdbindexes', 'plan_parcels_open.gdb/a00000006.freelist', 'plan_parcels_open.gdb/a00000005.CatItemTypesByName.atx', 'plan_parcels_open.gdb/a00000004.horizon', 'pl

In [5]:
mn_gsc_api_url = "https://gisdata.mn.gov/api/3/action"

# MN geospatial commons api search datasets request for counties

search_datasets_url = mn_gsc_api_url + "/package_search"
query = "County Boundaries, Minnesota"
request_url = search_datasets_url + f"?q={query}"
response = requests.get(request_url, verify=False) # verify=False disables SSL certification validation which makes it mad but i cant get it to work without disabling it
response_data = json.loads(response.text)
found_dataset_names = [dataset["title"] for dataset in response_data["result"]["results"]]
print(found_dataset_names)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['County Boundaries in Minnesota', 'County Boundaries, Minnesota', 'Municipal Boundaries, Dakota County, Minnesota', 'Administrative Boundaries, Ramsey County, Minnesota', 'Political and Administrative Boundaries, Dakota County, Minnesota', 'State Boundary, Minnesota', 'Boundaries of Minnesota', 'Senate District Boundaries, Minnesota, 2012', 'House District Boundaries, Minnesota, 1994', 'House District Boundaries, Minnesota, 2002']


In [6]:
# MN geospatial commons api get dataset request for counties

first_dataset_result_id = response_data["result"]["results"][1]["id"] # grab dataset that is second in returned dataset results
get_dataset_url = mn_gsc_api_url + "/package_show"
id = first_dataset_result_id
request_url = get_dataset_url + f"?id={id}"
response = requests.get(request_url, verify=False) # again disabling ssl cert validation
response_data = json.loads(response.text)
resources = response_data["result"]["resources"]
raw_zip_dataset_file_download_url = [resource for resource in resources if resource["resource_type"] == "fgdb"][0]["url"]
print(raw_zip_dataset_file_download_url)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/fgdb_bdry_counties_in_minnesota.zip


In [7]:
# download counties 
response = requests.get(raw_zip_dataset_file_download_url)
# check response.status_code to see if download was successful, should return '200'
os.chdir(r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject")
if response.status_code == 200:
    # Open zipfile
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        # extract contents of zipfile
        z.extractall('counties') # extracts contents into a folder named 'counties'
        
        # assuming there's only one counties file in the zip, get its name
        counties_file = [name for name in z.namelist()]
        print(counties_file)

['Counties in Minnesota.qlr', 'Counties in Minnesota.lyr', 'metadata/metadata.html', 'metadata/metadata.xml', 'bdry_counties_in_minnesota.gdb/a0000000e.gdbtable', 'bdry_counties_in_minnesota.gdb/a0000000d.gdbtablx', 'bdry_counties_in_minnesota.gdb/a0000000e.gdbindexes', 'bdry_counties_in_minnesota.gdb/a00000006.CatRelTypesByBackwardLabel.atx', 'bdry_counties_in_minnesota.gdb/a00000004.gdbtable', 'bdry_counties_in_minnesota.gdb/a0000000c.horizon', 'bdry_counties_in_minnesota.gdb/a00000006.gdbindexes', 'bdry_counties_in_minnesota.gdb/a00000007.gdbindexes', 'bdry_counties_in_minnesota.gdb/a0000000c.gdbtablx', 'bdry_counties_in_minnesota.gdb/a00000004.CatItemsByType.atx', 'bdry_counties_in_minnesota.gdb/a00000006.CatRelTypesByOriginItemTypeID.atx', 'bdry_counties_in_minnesota.gdb/a00000005.FDO_UUID.atx', 'bdry_counties_in_minnesota.gdb/a00000007.gdbtable', 'bdry_counties_in_minnesota.gdb/a0000000d.spx', 'bdry_counties_in_minnesota.gdb/a00000009.gdbtablx', 'bdry_counties_in_minnesota.gdb/a0

In [8]:
# export out dakota county
arcpy.conversion.ExportFeatures("mn_county_boundaries", r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject\counties", "CTY_ABBR = 'DAKO'")

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\counties.shp'>

In [10]:
# clip parcels to dakota county

arcpy.analysis.Clip(
    in_features="plan_parcels_open",
    clip_features="counties",
    out_feature_class=r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\DakotaParcels",
    cluster_tolerance=None
)

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\DakotaParcels'>

In [11]:
mn_gsc_api_url = "https://gisdata.mn.gov/api/3/action"

# MN geospatial commons api search datasets request for prairies

search_datasets_url = mn_gsc_api_url + "/package_search"
query = "MNDNR Native Prairies"
request_url = search_datasets_url + f"?q={query}"
response = requests.get(request_url, verify=False) # verify=False disables SSL certification validation which makes it mad but i cant get it to work without disabling it
response_data = json.loads(response.text)
found_dataset_names = [dataset["title"] for dataset in response_data["result"]["results"]]
print(found_dataset_names)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['MNDNR Native Prairies', 'MNDNR Native Plant Communities', 'MNDNR Native Plant Communities - Groundwater Subset', 'Prairie Change Analysis 1991-2008', 'MCBS Railroad Rights-of-Way Prairies', 'MNDNR Forest Inventory Old Growth', 'MNDNR SNA Conservation Opportunity Areas and Marxan Conservation Prioritization', 'Map displaying LSOHC Completed Project Locations', 'Potentially Undisturbed Land (Virgin Sod) - FSA Common Land Unit Derived', 'Scientific and Natural Area Units']


In [12]:
# MN geospatial commons api get dataset request for prairies

first_dataset_result_id = response_data["result"]["results"][0]["id"] # grab dataset that is first in returned dataset results
get_dataset_url = mn_gsc_api_url + "/package_show"
id = first_dataset_result_id
request_url = get_dataset_url + f"?id={id}"
response = requests.get(request_url, verify=False) # again disabling ssl cert validation
response_data = json.loads(response.text)
resources = response_data["result"]["resources"]
raw_zip_dataset_file_download_url = [resource for resource in resources if resource["resource_type"] == "fgdb"][0]["url"]
print(raw_zip_dataset_file_download_url)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_dnr_native_prairies/fgdb_biota_dnr_native_prairies.zip


In [13]:
# download prairies
response = requests.get(raw_zip_dataset_file_download_url)
# check response.status_code to see if download was successful, should return '200'
os.chdir(r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject")
if response.status_code == 200:
    # Open zipfile
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        # extract contents of zipfile
        z.extractall('prairies') # extracts contents into a folder named 'prairies'
        
        # assuming there's only one landuse file in the zip, get its name
        prairies_file = [name for name in z.namelist()]
        print(prairies_file)

['DNR Native Plant Communities - Prairie subset.lyr', 'metadata/S_and_G_ranks.pdf', 'metadata/metadata.html', 'metadata/MN xwalk with NVC 2.0 hierarchy_9may16.xlsx', 'metadata/S_and_G_ranks.html', 'metadata/metadata.xml', 'metadata/NPC_Complexes.pdf', 'metadata/NPC_Complexes.html', 'metadata/S_and_G_ranks.xls', 'metadata/NPC_Complexes.xlsx', 'biota_dnr_native_prairies.gdb/a0000099e.gdbtable', 'biota_dnr_native_prairies.gdb/a00000004.gdbtable', 'biota_dnr_native_prairies.gdb/a00000006.gdbindexes', 'biota_dnr_native_prairies.gdb/a00000296.gdbtable', 'biota_dnr_native_prairies.gdb/a00000007.gdbindexes', 'biota_dnr_native_prairies.gdb/a00000007.CatRelTypesByBackwardLabel.atx', 'biota_dnr_native_prairies.gdb/a00000297.gdbtablx', 'biota_dnr_native_prairies.gdb/a00000004.CatItemsByType.atx', 'biota_dnr_native_prairies.gdb/a00000007.CatRelTypesByForwardLabel.atx', 'biota_dnr_native_prairies.gdb/a00000007.gdbtable', 'biota_dnr_native_prairies.gdb/a00000295.gdbtablx', 'biota_dnr_native_prairies.

In [3]:
# clip out dakota county native prairies

arcpy.analysis.Clip(
    in_features="dnr_native_prairies",
    clip_features="DakotaParcels",
    out_feature_class=r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\DakotaPrairies",
    cluster_tolerance=None
)

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\DakotaPrairies'>

In [4]:
# get numpy array of parcel attribute table

arcpy.env.workspace = r'C:\Users\18284\Documents\ArcGIS\Projects\finalproject'
shapefile = 'DakotaParcels'

field_names = [field.name for field in arcpy.ListFields(shapefile)]

attribute_array = arcpy.da.TableToNumPyArray(shapefile, field_names)

print(attribute_array)

ValueError: cannot create NumPyArray. geometry type found

In [9]:
# MAIN SETUP FOR CONSTANTS

import pandas as pd
from sklearn.neighbors import NearestNeighbors

all_fields = ['OID@', 'objectid', 'county_pin', 'state_pin', 'own_add_l1', 'tax_name', 'acres_poly', 'emv_land', 'emv_bldg', 'emv_total', 'useclass1', 'useclass2','SHAPE@TRUECENTROID']
fc = 'DakotaParcels'
all_fields2 = ['OID@','SHAPE@TRUECENTROID']
fc2 = 'DakotaPrairies'

#  A) LOAD PARCEL AND PRARIE DATA FROM SHAPEFILES
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# LOAD PARCEL DATA TO NP ARRAY
parcel_numpy = arcpy.da.FeatureClassToNumPyArray(fc, all_fields, "")
num_parcels = len(parcel_numpy)

# LOAD PRARIE DATA TO NP ARRAY
prairie_numpy = arcpy.da.FeatureClassToNumPyArray(fc2, all_fields2, "")
num_prairies = len(parcel_numpy)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#  B) MAP LAND ZONING TO AN INTEGER VALUE FOR COMPUTATIONAL ANALYSIS
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# CREATE A USE CASE MAP
class_options_string = numpy.unique(parcel_numpy['useclass1'])
class_options_sorted = numpy.argsort(class_options_string) # TEMPORARY MANUALLY SET THIS L8R (PRIORITY OF LAND USE)
class_options_id = numpy.arange(0, len(class_options_string))[class_options_sorted]
class_options_string = class_options_string[class_options_sorted]
class_map = dict(zip(class_options_string, class_options_id))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#  C) SEPERATE OUT DATA FOR THE PARCELS ALREADY OWNED BY THE FISH & WILDLIFE SERVICE OR
#            THE DNR BY CREATING MASKING ARRAYS
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# CHECK FOR DNR OR FWS LAND AND CREATE FILTER
isFWSLand = numpy.zeros(num_parcels)
isDNRLand = numpy.zeros(num_parcels)

# ITERATOR TO CREATE VARIOUS PARCEL VARIABLE
for i in range(num_parcels):
    parcel_numpy['useclass2'][i] = class_map[parcel_numpy['useclass1'][i]]
    isFWSLand[i] = (('fish' in (parcel_numpy['own_add_l1'][i]).lower()) 
                    and ('wildlife' in (parcel_numpy['own_add_l1'][i]).lower())) or (
                    ('fish' in (parcel_numpy['tax_name'][i]).lower()) 
                    and ('wildlife' in (parcel_numpy['tax_name'][i]).lower()))
    isDNRLand[i] = ('dnr' in (parcel_numpy['tax_name'][i]).lower())

# PROSPECTIVE NON-GOVERNMENT LAND
#    IF PARCEL IS NOT FWSLAND OR DNRLAND ADD IT TO THE PROSPECTIVE PARCELS
isProspect = numpy.logical_not((isFWSLand+isDNRLand).astype(bool)).astype(bool)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#  D) APPLY THE MASKING ARRAYS TO CREATE DATAFRAMES OF THE DNR, FWS, AND PROSPECTIVE PARCELS
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# GENERATE PANDAS DATAFRAMES FOR DATA MANAGEMENT
dfFWSParcels = pd.DataFrame(parcel_numpy[isFWSLand.astype(bool)].tolist(), columns=all_fields)
dfDNRParcels = pd.DataFrame(parcel_numpy[isDNRLand.astype(bool)].tolist(), columns=all_fields)
dfProspectiveParcels = pd.DataFrame(parcel_numpy[isProspect.astype(bool)].tolist(), columns=all_fields)
dfProspectiveParcels['useclass2'] = pd.to_numeric(dfProspectiveParcels['useclass2'])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#  E) CALCULATE AND ADD THE DISTANCE TO THE NEAREST PRAIRIE TO THE DATAFRAMES
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dfPrairies = pd.DataFrame(prairie_numpy.tolist(), columns=all_fields2)
# COMPUTE THE CENTROIDS
FWSCentroids = numpy.array(dfDNRParcels['SHAPE@TRUECENTROID'].tolist())
DNRCentroids = numpy.array(dfFWSParcels['SHAPE@TRUECENTROID'].tolist())
PrairieCentroids = numpy.array(dfPrairies['SHAPE@TRUECENTROID'].tolist())
AllCentroids = numpy.append(numpy.append(FWSCentroids, DNRCentroids, axis=0), PrairieCentroids, axis=0)
ProspectCentroids = numpy.array(dfProspectiveParcels['SHAPE@TRUECENTROID'].tolist())

# USE SCIKIT LEARN'S NEAREST NEIGHBOR FUNCTION TO DETERMINE DISTANCES
FWSnbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(FWSCentroids)
DNRnbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(DNRCentroids)
PRAIRIEnbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(PrairieCentroids)
ALLnbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(AllCentroids)
dfProspectiveParcels['distance_fws_NN'], dfProspectiveParcels['index_fws_NN'] = FWSnbrs.kneighbors(ProspectCentroids)
dfProspectiveParcels['distance_dnr_NN'], dfProspectiveParcels['index_dnr_NN'] = DNRnbrs.kneighbors(ProspectCentroids)
dfProspectiveParcels['distance_prairie_NN'], dfProspectiveParcels['index_prairie_NN'] = PRAIRIEnbrs.kneighbors(ProspectCentroids)
dfProspectiveParcels['distance_all_NN'], dfProspectiveParcels['index_all_NN'] = ALLnbrs.kneighbors(ProspectCentroids)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#  F) CALCULATE AND ADD THE DOLLARS PER ACRE TO THE PROSPECTIVE PARCELS DATAFRAME
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# CALCULATE DOLLARS PER ACRES POLY
dfProspectiveParcels['usd_per_acre_total'] = numpy.divide(dfProspectiveParcels['emv_total'], dfProspectiveParcels['acres_poly'])
dfProspectiveParcels['usd_per_acre_bldg'] = numpy.divide(dfProspectiveParcels['emv_bldg'], dfProspectiveParcels['acres_poly'])
dfProspectiveParcels['usd_per_acre_land'] = numpy.divide(dfProspectiveParcels['emv_land'], dfProspectiveParcels['acres_poly'])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#   G) HUMAN READABLE DATA HAS BEEN GENERATED!

In [10]:
# EXPORTS DAKOTA PARCELS DATA to FWS and DNR LAYERS


for parcelOID in dfFWSParcels['objectid'].tolist():
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="DakotaParcels",
        selection_type="ADD_TO_SELECTION",
        where_clause="objectid = "+ str(parcelOID),
        invert_where_clause=None
    )

arcpy.conversion.ExportFeatures(
    in_features="DakotaParcels",
    out_features="DakotaParcels_FWS",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='objectid "objectid" true true false 4 Long 0 0,First,#,DakotaParcels,objectid,-1,-1;county_pin "county_pin" true true false 22 Text 0 0,First,#,DakotaParcels,county_pin,0,22;state_pin "state_pin" true true false 28 Text 0 0,First,#,DakotaParcels,state_pin,0,28;anumberpre "anumberpre" true true false 15 Text 0 0,First,#,DakotaParcels,anumberpre,0,15;anumber "anumber" true true false 4 Long 0 0,First,#,DakotaParcels,anumber,-1,-1;anumbersuf "anumbersuf" true true false 15 Text 0 0,First,#,DakotaParcels,anumbersuf,0,15;st_pre_mod "st_pre_mod" true true false 15 Text 0 0,First,#,DakotaParcels,st_pre_mod,0,15;st_pre_dir "st_pre_dir" true true false 9 Text 0 0,First,#,DakotaParcels,st_pre_dir,0,9;st_pre_typ "st_pre_typ" true true false 35 Text 0 0,First,#,DakotaParcels,st_pre_typ,0,35;st_pre_sep "st_pre_sep" true true false 20 Text 0 0,First,#,DakotaParcels,st_pre_sep,0,20;st_name "st_name" true true false 60 Text 0 0,First,#,DakotaParcels,st_name,0,60;st_pos_typ "st_pos_typ" true true false 15 Text 0 0,First,#,DakotaParcels,st_pos_typ,0,15;st_pos_dir "st_pos_dir" true true false 9 Text 0 0,First,#,DakotaParcels,st_pos_dir,0,9;st_pos_mod "st_pos_mod" true true false 15 Text 0 0,First,#,DakotaParcels,st_pos_mod,0,15;sub_type1 "sub_type1" true true false 12 Text 0 0,First,#,DakotaParcels,sub_type1,0,12;sub_id1 "sub_id1" true true false 30 Text 0 0,First,#,DakotaParcels,sub_id1,0,30;sub_type2 "sub_type2" true true false 12 Text 0 0,First,#,DakotaParcels,sub_type2,0,12;sub_id2 "sub_id2" true true false 30 Text 0 0,First,#,DakotaParcels,sub_id2,0,30;zip "zip" true true false 5 Text 0 0,First,#,DakotaParcels,zip,0,5;zip4 "zip4" true true false 4 Text 0 0,First,#,DakotaParcels,zip4,0,4;ctu_name "ctu_name" true true false 100 Text 0 0,First,#,DakotaParcels,ctu_name,0,100;ctu_id_txt "ctu_id_txt" true true false 8 Text 0 0,First,#,DakotaParcels,ctu_id_txt,0,8;postcomm "postcomm" true true false 40 Text 0 0,First,#,DakotaParcels,postcomm,0,40;co_code "co_code" true true false 5 Text 0 0,First,#,DakotaParcels,co_code,0,5;co_name "co_name" true true false 40 Text 0 0,First,#,DakotaParcels,co_name,0,40;state_code "state_code" true true false 2 Text 0 0,First,#,DakotaParcels,state_code,0,2;lot "lot" true true false 30 Text 0 0,First,#,DakotaParcels,lot,0,30;block "block" true true false 30 Text 0 0,First,#,DakotaParcels,block,0,30;plat_name "plat_name" true true false 150 Text 0 0,First,#,DakotaParcels,plat_name,0,150;owner_name "owner_name" true true false 100 Text 0 0,First,#,DakotaParcels,owner_name,0,100;owner_more "owner_more" true true false 100 Text 0 0,First,#,DakotaParcels,owner_more,0,100;own_add_l1 "own_add_l1" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l1,0,100;own_add_l2 "own_add_l2" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l2,0,100;own_add_l3 "own_add_l3" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l3,0,100;own_add_l4 "own_add_l4" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l4,0,100;tax_name "tax_name" true true false 100 Text 0 0,First,#,DakotaParcels,tax_name,0,100;tax_add_l1 "tax_add_l1" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l1,0,100;tax_add_l2 "tax_add_l2" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l2,0,100;tax_add_l3 "tax_add_l3" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l3,0,100;tax_add_l4 "tax_add_l4" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l4,0,100;landmark "landmark" true true false 150 Text 0 0,First,#,DakotaParcels,landmark,0,150;homestead "homestead" true true false 10 Text 0 0,First,#,DakotaParcels,homestead,0,10;acres_poly "acres_poly" true true false 8 Double 0 0,First,#,DakotaParcels,acres_poly,-1,-1;acres_deed "acres_deed" true true false 8 Double 0 0,First,#,DakotaParcels,acres_deed,-1,-1;emv_land "emv_land" true true false 4 Long 0 0,First,#,DakotaParcels,emv_land,-1,-1;emv_bldg "emv_bldg" true true false 4 Long 0 0,First,#,DakotaParcels,emv_bldg,-1,-1;emv_total "emv_total" true true false 4 Long 0 0,First,#,DakotaParcels,emv_total,-1,-1;tax_year "tax_year" true true false 2 Short 0 0,First,#,DakotaParcels,tax_year,-1,-1;mkt_year "mkt_year" true true false 2 Short 0 0,First,#,DakotaParcels,mkt_year,-1,-1;tax_capac "tax_capac" true true false 4 Long 0 0,First,#,DakotaParcels,tax_capac,-1,-1;total_tax "total_tax" true true false 4 Long 0 0,First,#,DakotaParcels,total_tax,-1,-1;spec_asses "spec_asses" true true false 4 Long 0 0,First,#,DakotaParcels,spec_asses,-1,-1;useclass1 "useclass1" true true false 100 Text 0 0,First,#,DakotaParcels,useclass1,0,100;useclass2 "useclass2" true true false 100 Text 0 0,First,#,DakotaParcels,useclass2,0,100;useclass3 "useclass3" true true false 100 Text 0 0,First,#,DakotaParcels,useclass3,0,100;useclass4 "useclass4" true true false 100 Text 0 0,First,#,DakotaParcels,useclass4,0,100;multi_uses "multi_uses" true true false 10 Text 0 0,First,#,DakotaParcels,multi_uses,0,10;tax_exempt "tax_exempt" true true false 3 Text 0 0,First,#,DakotaParcels,tax_exempt,0,3;xuseclass1 "xuseclass1" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass1,0,100;xuseclass2 "xuseclass2" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass2,0,100;xuseclass3 "xuseclass3" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass3,0,100;xuseclass4 "xuseclass4" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass4,0,100;dwell_type "dwell_type" true true false 30 Text 0 0,First,#,DakotaParcels,dwell_type,0,30;home_style "home_style" true true false 30 Text 0 0,First,#,DakotaParcels,home_style,0,30;fin_sq_ft "fin_sq_ft" true true false 4 Long 0 0,First,#,DakotaParcels,fin_sq_ft,-1,-1;garage "garage" true true false 10 Text 0 0,First,#,DakotaParcels,garage,0,10;garagesqft "garagesqft" true true false 4 Long 0 0,First,#,DakotaParcels,garagesqft,-1,-1;basement "basement" true true false 10 Text 0 0,First,#,DakotaParcels,basement,0,10;heating "heating" true true false 30 Text 0 0,First,#,DakotaParcels,heating,0,30;cooling "cooling" true true false 30 Text 0 0,First,#,DakotaParcels,cooling,0,30;year_built "year_built" true true false 2 Short 0 0,First,#,DakotaParcels,year_built,-1,-1;num_units "num_units" true true false 4 Long 0 0,First,#,DakotaParcels,num_units,-1,-1;sale_date "sale_date" true true false 8 Date 0 0,First,#,DakotaParcels,sale_date,-1,-1;sale_value "sale_value" true true false 4 Long 0 0,First,#,DakotaParcels,sale_value,-1,-1;green_acre "green_acre" true true false 10 Text 0 0,First,#,DakotaParcels,green_acre,0,10;open_space "open_space" true true false 10 Text 0 0,First,#,DakotaParcels,open_space,0,10;ag_preserv "ag_preserv" true true false 10 Text 0 0,First,#,DakotaParcels,ag_preserv,0,10;agpre_enrd "agpre_enrd" true true false 8 Date 0 0,First,#,DakotaParcels,agpre_enrd,-1,-1;agpre_expd "agpre_expd" true true false 8 Date 0 0,First,#,DakotaParcels,agpre_expd,-1,-1;abb_legal "abb_legal" true true false 254 Text 0 0,First,#,DakotaParcels,abb_legal,0,254;edit_date "edit_date" true true false 8 Date 0 0,First,#,DakotaParcels,edit_date,-1,-1;exp_date "exp_date" true true false 8 Date 0 0,First,#,DakotaParcels,exp_date,-1,-1;polyptrel "polyptrel" true true false 2 Short 0 0,First,#,DakotaParcels,polyptrel,-1,-1;n_standard "n_standard" true true false 2 Short 0 0,First,#,DakotaParcels,n_standard,-1,-1;ownership "ownership" true true false 30 Text 0 0,First,#,DakotaParcels,ownership,0,30;school_dst "school_dst" true true false 10 Text 0 0,First,#,DakotaParcels,school_dst,0,10;wshd_dst "wshd_dst" true true false 50 Text 0 0,First,#,DakotaParcels,wshd_dst,0,50;section "section" true true false 2 Short 0 0,First,#,DakotaParcels,section,-1,-1;township "township" true true false 2 Short 0 0,First,#,DakotaParcels,township,-1,-1;range "range" true true false 2 Short 0 0,First,#,DakotaParcels,range,-1,-1;range_dir "range_dir" true true false 2 Short 0 0,First,#,DakotaParcels,range_dir,-1,-1;prin_mer "prin_mer" true true false 2 Short 0 0,First,#,DakotaParcels,prin_mer,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,DakotaParcels,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,DakotaParcels,Shape_Area,-1,-1',
    sort_field=None
)

for parcelOID in dfDNRParcels['objectid'].tolist():
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="DakotaParcels",
        selection_type="ADD_TO_SELECTION",
        where_clause="objectid = "+ str(parcelOID),
        invert_where_clause=None
    )

arcpy.conversion.ExportFeatures(
    in_features="DakotaParcels",
    out_features="DakotaParcels_FWSDNR",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='objectid "objectid" true true false 4 Long 0 0,First,#,DakotaParcels,objectid,-1,-1;county_pin "county_pin" true true false 22 Text 0 0,First,#,DakotaParcels,county_pin,0,22;state_pin "state_pin" true true false 28 Text 0 0,First,#,DakotaParcels,state_pin,0,28;anumberpre "anumberpre" true true false 15 Text 0 0,First,#,DakotaParcels,anumberpre,0,15;anumber "anumber" true true false 4 Long 0 0,First,#,DakotaParcels,anumber,-1,-1;anumbersuf "anumbersuf" true true false 15 Text 0 0,First,#,DakotaParcels,anumbersuf,0,15;st_pre_mod "st_pre_mod" true true false 15 Text 0 0,First,#,DakotaParcels,st_pre_mod,0,15;st_pre_dir "st_pre_dir" true true false 9 Text 0 0,First,#,DakotaParcels,st_pre_dir,0,9;st_pre_typ "st_pre_typ" true true false 35 Text 0 0,First,#,DakotaParcels,st_pre_typ,0,35;st_pre_sep "st_pre_sep" true true false 20 Text 0 0,First,#,DakotaParcels,st_pre_sep,0,20;st_name "st_name" true true false 60 Text 0 0,First,#,DakotaParcels,st_name,0,60;st_pos_typ "st_pos_typ" true true false 15 Text 0 0,First,#,DakotaParcels,st_pos_typ,0,15;st_pos_dir "st_pos_dir" true true false 9 Text 0 0,First,#,DakotaParcels,st_pos_dir,0,9;st_pos_mod "st_pos_mod" true true false 15 Text 0 0,First,#,DakotaParcels,st_pos_mod,0,15;sub_type1 "sub_type1" true true false 12 Text 0 0,First,#,DakotaParcels,sub_type1,0,12;sub_id1 "sub_id1" true true false 30 Text 0 0,First,#,DakotaParcels,sub_id1,0,30;sub_type2 "sub_type2" true true false 12 Text 0 0,First,#,DakotaParcels,sub_type2,0,12;sub_id2 "sub_id2" true true false 30 Text 0 0,First,#,DakotaParcels,sub_id2,0,30;zip "zip" true true false 5 Text 0 0,First,#,DakotaParcels,zip,0,5;zip4 "zip4" true true false 4 Text 0 0,First,#,DakotaParcels,zip4,0,4;ctu_name "ctu_name" true true false 100 Text 0 0,First,#,DakotaParcels,ctu_name,0,100;ctu_id_txt "ctu_id_txt" true true false 8 Text 0 0,First,#,DakotaParcels,ctu_id_txt,0,8;postcomm "postcomm" true true false 40 Text 0 0,First,#,DakotaParcels,postcomm,0,40;co_code "co_code" true true false 5 Text 0 0,First,#,DakotaParcels,co_code,0,5;co_name "co_name" true true false 40 Text 0 0,First,#,DakotaParcels,co_name,0,40;state_code "state_code" true true false 2 Text 0 0,First,#,DakotaParcels,state_code,0,2;lot "lot" true true false 30 Text 0 0,First,#,DakotaParcels,lot,0,30;block "block" true true false 30 Text 0 0,First,#,DakotaParcels,block,0,30;plat_name "plat_name" true true false 150 Text 0 0,First,#,DakotaParcels,plat_name,0,150;owner_name "owner_name" true true false 100 Text 0 0,First,#,DakotaParcels,owner_name,0,100;owner_more "owner_more" true true false 100 Text 0 0,First,#,DakotaParcels,owner_more,0,100;own_add_l1 "own_add_l1" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l1,0,100;own_add_l2 "own_add_l2" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l2,0,100;own_add_l3 "own_add_l3" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l3,0,100;own_add_l4 "own_add_l4" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l4,0,100;tax_name "tax_name" true true false 100 Text 0 0,First,#,DakotaParcels,tax_name,0,100;tax_add_l1 "tax_add_l1" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l1,0,100;tax_add_l2 "tax_add_l2" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l2,0,100;tax_add_l3 "tax_add_l3" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l3,0,100;tax_add_l4 "tax_add_l4" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l4,0,100;landmark "landmark" true true false 150 Text 0 0,First,#,DakotaParcels,landmark,0,150;homestead "homestead" true true false 10 Text 0 0,First,#,DakotaParcels,homestead,0,10;acres_poly "acres_poly" true true false 8 Double 0 0,First,#,DakotaParcels,acres_poly,-1,-1;acres_deed "acres_deed" true true false 8 Double 0 0,First,#,DakotaParcels,acres_deed,-1,-1;emv_land "emv_land" true true false 4 Long 0 0,First,#,DakotaParcels,emv_land,-1,-1;emv_bldg "emv_bldg" true true false 4 Long 0 0,First,#,DakotaParcels,emv_bldg,-1,-1;emv_total "emv_total" true true false 4 Long 0 0,First,#,DakotaParcels,emv_total,-1,-1;tax_year "tax_year" true true false 2 Short 0 0,First,#,DakotaParcels,tax_year,-1,-1;mkt_year "mkt_year" true true false 2 Short 0 0,First,#,DakotaParcels,mkt_year,-1,-1;tax_capac "tax_capac" true true false 4 Long 0 0,First,#,DakotaParcels,tax_capac,-1,-1;total_tax "total_tax" true true false 4 Long 0 0,First,#,DakotaParcels,total_tax,-1,-1;spec_asses "spec_asses" true true false 4 Long 0 0,First,#,DakotaParcels,spec_asses,-1,-1;useclass1 "useclass1" true true false 100 Text 0 0,First,#,DakotaParcels,useclass1,0,100;useclass2 "useclass2" true true false 100 Text 0 0,First,#,DakotaParcels,useclass2,0,100;useclass3 "useclass3" true true false 100 Text 0 0,First,#,DakotaParcels,useclass3,0,100;useclass4 "useclass4" true true false 100 Text 0 0,First,#,DakotaParcels,useclass4,0,100;multi_uses "multi_uses" true true false 10 Text 0 0,First,#,DakotaParcels,multi_uses,0,10;tax_exempt "tax_exempt" true true false 3 Text 0 0,First,#,DakotaParcels,tax_exempt,0,3;xuseclass1 "xuseclass1" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass1,0,100;xuseclass2 "xuseclass2" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass2,0,100;xuseclass3 "xuseclass3" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass3,0,100;xuseclass4 "xuseclass4" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass4,0,100;dwell_type "dwell_type" true true false 30 Text 0 0,First,#,DakotaParcels,dwell_type,0,30;home_style "home_style" true true false 30 Text 0 0,First,#,DakotaParcels,home_style,0,30;fin_sq_ft "fin_sq_ft" true true false 4 Long 0 0,First,#,DakotaParcels,fin_sq_ft,-1,-1;garage "garage" true true false 10 Text 0 0,First,#,DakotaParcels,garage,0,10;garagesqft "garagesqft" true true false 4 Long 0 0,First,#,DakotaParcels,garagesqft,-1,-1;basement "basement" true true false 10 Text 0 0,First,#,DakotaParcels,basement,0,10;heating "heating" true true false 30 Text 0 0,First,#,DakotaParcels,heating,0,30;cooling "cooling" true true false 30 Text 0 0,First,#,DakotaParcels,cooling,0,30;year_built "year_built" true true false 2 Short 0 0,First,#,DakotaParcels,year_built,-1,-1;num_units "num_units" true true false 4 Long 0 0,First,#,DakotaParcels,num_units,-1,-1;sale_date "sale_date" true true false 8 Date 0 0,First,#,DakotaParcels,sale_date,-1,-1;sale_value "sale_value" true true false 4 Long 0 0,First,#,DakotaParcels,sale_value,-1,-1;green_acre "green_acre" true true false 10 Text 0 0,First,#,DakotaParcels,green_acre,0,10;open_space "open_space" true true false 10 Text 0 0,First,#,DakotaParcels,open_space,0,10;ag_preserv "ag_preserv" true true false 10 Text 0 0,First,#,DakotaParcels,ag_preserv,0,10;agpre_enrd "agpre_enrd" true true false 8 Date 0 0,First,#,DakotaParcels,agpre_enrd,-1,-1;agpre_expd "agpre_expd" true true false 8 Date 0 0,First,#,DakotaParcels,agpre_expd,-1,-1;abb_legal "abb_legal" true true false 254 Text 0 0,First,#,DakotaParcels,abb_legal,0,254;edit_date "edit_date" true true false 8 Date 0 0,First,#,DakotaParcels,edit_date,-1,-1;exp_date "exp_date" true true false 8 Date 0 0,First,#,DakotaParcels,exp_date,-1,-1;polyptrel "polyptrel" true true false 2 Short 0 0,First,#,DakotaParcels,polyptrel,-1,-1;n_standard "n_standard" true true false 2 Short 0 0,First,#,DakotaParcels,n_standard,-1,-1;ownership "ownership" true true false 30 Text 0 0,First,#,DakotaParcels,ownership,0,30;school_dst "school_dst" true true false 10 Text 0 0,First,#,DakotaParcels,school_dst,0,10;wshd_dst "wshd_dst" true true false 50 Text 0 0,First,#,DakotaParcels,wshd_dst,0,50;section "section" true true false 2 Short 0 0,First,#,DakotaParcels,section,-1,-1;township "township" true true false 2 Short 0 0,First,#,DakotaParcels,township,-1,-1;range "range" true true false 2 Short 0 0,First,#,DakotaParcels,range,-1,-1;range_dir "range_dir" true true false 2 Short 0 0,First,#,DakotaParcels,range_dir,-1,-1;prin_mer "prin_mer" true true false 2 Short 0 0,First,#,DakotaParcels,prin_mer,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,DakotaParcels,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,DakotaParcels,Shape_Area,-1,-1',
    sort_field=None
)


<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\DakotaParcels_FWSDNR.shp'>

In [33]:
# DO NOT!
# DO NOT!
# DO NOT RUN UNLESS YOU WANT TO RESET THE NEGATIVE TRAINING DATA
all_fields = ['OID@', 'objectid', 'county_pin', 'state_pin', 'own_add_l1', 'tax_name', 'acres_poly', 'emv_land', 'emv_bldg', 'emv_total', 'useclass1', 'useclass2','SHAPE@TRUECENTROID']


for parcelOID in dfProspectiveParcels.sample(n = 25)['objectid'].tolist():
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="DakotaParcels",
        selection_type="ADD_TO_SELECTION",
        where_clause="objectid = "+ str(parcelOID),
        invert_where_clause=None)

dakotaparcel_positivenumpy = arcpy.da.FeatureClassToNumPyArray("DakotaParcels_trainingnegative", all_fields, "")

In [32]:
for parcelOID in dakotaparcel_positivenumpy['objectid'].tolist():
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="DakotaParcels",
        selection_type="ADD_TO_SELECTION",
        where_clause="objectid = "+ str(parcelOID),
        invert_where_clause=None)

arcpy.conversion.ExportFeatures(
    in_features="DakotaParcels",
    out_features="DakotaParcels_trainingnegative",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='objectid "objectid" true true false 4 Long 0 0,First,#,DakotaParcels,objectid,-1,-1;county_pin "county_pin" true true false 22 Text 0 0,First,#,DakotaParcels,county_pin,0,22;state_pin "state_pin" true true false 28 Text 0 0,First,#,DakotaParcels,state_pin,0,28;anumberpre "anumberpre" true true false 15 Text 0 0,First,#,DakotaParcels,anumberpre,0,15;anumber "anumber" true true false 4 Long 0 0,First,#,DakotaParcels,anumber,-1,-1;anumbersuf "anumbersuf" true true false 15 Text 0 0,First,#,DakotaParcels,anumbersuf,0,15;st_pre_mod "st_pre_mod" true true false 15 Text 0 0,First,#,DakotaParcels,st_pre_mod,0,15;st_pre_dir "st_pre_dir" true true false 9 Text 0 0,First,#,DakotaParcels,st_pre_dir,0,9;st_pre_typ "st_pre_typ" true true false 35 Text 0 0,First,#,DakotaParcels,st_pre_typ,0,35;st_pre_sep "st_pre_sep" true true false 20 Text 0 0,First,#,DakotaParcels,st_pre_sep,0,20;st_name "st_name" true true false 60 Text 0 0,First,#,DakotaParcels,st_name,0,60;st_pos_typ "st_pos_typ" true true false 15 Text 0 0,First,#,DakotaParcels,st_pos_typ,0,15;st_pos_dir "st_pos_dir" true true false 9 Text 0 0,First,#,DakotaParcels,st_pos_dir,0,9;st_pos_mod "st_pos_mod" true true false 15 Text 0 0,First,#,DakotaParcels,st_pos_mod,0,15;sub_type1 "sub_type1" true true false 12 Text 0 0,First,#,DakotaParcels,sub_type1,0,12;sub_id1 "sub_id1" true true false 30 Text 0 0,First,#,DakotaParcels,sub_id1,0,30;sub_type2 "sub_type2" true true false 12 Text 0 0,First,#,DakotaParcels,sub_type2,0,12;sub_id2 "sub_id2" true true false 30 Text 0 0,First,#,DakotaParcels,sub_id2,0,30;zip "zip" true true false 5 Text 0 0,First,#,DakotaParcels,zip,0,5;zip4 "zip4" true true false 4 Text 0 0,First,#,DakotaParcels,zip4,0,4;ctu_name "ctu_name" true true false 100 Text 0 0,First,#,DakotaParcels,ctu_name,0,100;ctu_id_txt "ctu_id_txt" true true false 8 Text 0 0,First,#,DakotaParcels,ctu_id_txt,0,8;postcomm "postcomm" true true false 40 Text 0 0,First,#,DakotaParcels,postcomm,0,40;co_code "co_code" true true false 5 Text 0 0,First,#,DakotaParcels,co_code,0,5;co_name "co_name" true true false 40 Text 0 0,First,#,DakotaParcels,co_name,0,40;state_code "state_code" true true false 2 Text 0 0,First,#,DakotaParcels,state_code,0,2;lot "lot" true true false 30 Text 0 0,First,#,DakotaParcels,lot,0,30;block "block" true true false 30 Text 0 0,First,#,DakotaParcels,block,0,30;plat_name "plat_name" true true false 150 Text 0 0,First,#,DakotaParcels,plat_name,0,150;owner_name "owner_name" true true false 100 Text 0 0,First,#,DakotaParcels,owner_name,0,100;owner_more "owner_more" true true false 100 Text 0 0,First,#,DakotaParcels,owner_more,0,100;own_add_l1 "own_add_l1" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l1,0,100;own_add_l2 "own_add_l2" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l2,0,100;own_add_l3 "own_add_l3" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l3,0,100;own_add_l4 "own_add_l4" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l4,0,100;tax_name "tax_name" true true false 100 Text 0 0,First,#,DakotaParcels,tax_name,0,100;tax_add_l1 "tax_add_l1" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l1,0,100;tax_add_l2 "tax_add_l2" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l2,0,100;tax_add_l3 "tax_add_l3" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l3,0,100;tax_add_l4 "tax_add_l4" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l4,0,100;landmark "landmark" true true false 150 Text 0 0,First,#,DakotaParcels,landmark,0,150;homestead "homestead" true true false 10 Text 0 0,First,#,DakotaParcels,homestead,0,10;acres_poly "acres_poly" true true false 8 Double 0 0,First,#,DakotaParcels,acres_poly,-1,-1;acres_deed "acres_deed" true true false 8 Double 0 0,First,#,DakotaParcels,acres_deed,-1,-1;emv_land "emv_land" true true false 4 Long 0 0,First,#,DakotaParcels,emv_land,-1,-1;emv_bldg "emv_bldg" true true false 4 Long 0 0,First,#,DakotaParcels,emv_bldg,-1,-1;emv_total "emv_total" true true false 4 Long 0 0,First,#,DakotaParcels,emv_total,-1,-1;tax_year "tax_year" true true false 2 Short 0 0,First,#,DakotaParcels,tax_year,-1,-1;mkt_year "mkt_year" true true false 2 Short 0 0,First,#,DakotaParcels,mkt_year,-1,-1;tax_capac "tax_capac" true true false 4 Long 0 0,First,#,DakotaParcels,tax_capac,-1,-1;total_tax "total_tax" true true false 4 Long 0 0,First,#,DakotaParcels,total_tax,-1,-1;spec_asses "spec_asses" true true false 4 Long 0 0,First,#,DakotaParcels,spec_asses,-1,-1;useclass1 "useclass1" true true false 100 Text 0 0,First,#,DakotaParcels,useclass1,0,100;useclass2 "useclass2" true true false 100 Text 0 0,First,#,DakotaParcels,useclass2,0,100;useclass3 "useclass3" true true false 100 Text 0 0,First,#,DakotaParcels,useclass3,0,100;useclass4 "useclass4" true true false 100 Text 0 0,First,#,DakotaParcels,useclass4,0,100;multi_uses "multi_uses" true true false 10 Text 0 0,First,#,DakotaParcels,multi_uses,0,10;tax_exempt "tax_exempt" true true false 3 Text 0 0,First,#,DakotaParcels,tax_exempt,0,3;xuseclass1 "xuseclass1" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass1,0,100;xuseclass2 "xuseclass2" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass2,0,100;xuseclass3 "xuseclass3" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass3,0,100;xuseclass4 "xuseclass4" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass4,0,100;dwell_type "dwell_type" true true false 30 Text 0 0,First,#,DakotaParcels,dwell_type,0,30;home_style "home_style" true true false 30 Text 0 0,First,#,DakotaParcels,home_style,0,30;fin_sq_ft "fin_sq_ft" true true false 4 Long 0 0,First,#,DakotaParcels,fin_sq_ft,-1,-1;garage "garage" true true false 10 Text 0 0,First,#,DakotaParcels,garage,0,10;garagesqft "garagesqft" true true false 4 Long 0 0,First,#,DakotaParcels,garagesqft,-1,-1;basement "basement" true true false 10 Text 0 0,First,#,DakotaParcels,basement,0,10;heating "heating" true true false 30 Text 0 0,First,#,DakotaParcels,heating,0,30;cooling "cooling" true true false 30 Text 0 0,First,#,DakotaParcels,cooling,0,30;year_built "year_built" true true false 2 Short 0 0,First,#,DakotaParcels,year_built,-1,-1;num_units "num_units" true true false 4 Long 0 0,First,#,DakotaParcels,num_units,-1,-1;sale_date "sale_date" true true false 8 Date 0 0,First,#,DakotaParcels,sale_date,-1,-1;sale_value "sale_value" true true false 4 Long 0 0,First,#,DakotaParcels,sale_value,-1,-1;green_acre "green_acre" true true false 10 Text 0 0,First,#,DakotaParcels,green_acre,0,10;open_space "open_space" true true false 10 Text 0 0,First,#,DakotaParcels,open_space,0,10;ag_preserv "ag_preserv" true true false 10 Text 0 0,First,#,DakotaParcels,ag_preserv,0,10;agpre_enrd "agpre_enrd" true true false 8 Date 0 0,First,#,DakotaParcels,agpre_enrd,-1,-1;agpre_expd "agpre_expd" true true false 8 Date 0 0,First,#,DakotaParcels,agpre_expd,-1,-1;abb_legal "abb_legal" true true false 254 Text 0 0,First,#,DakotaParcels,abb_legal,0,254;edit_date "edit_date" true true false 8 Date 0 0,First,#,DakotaParcels,edit_date,-1,-1;exp_date "exp_date" true true false 8 Date 0 0,First,#,DakotaParcels,exp_date,-1,-1;polyptrel "polyptrel" true true false 2 Short 0 0,First,#,DakotaParcels,polyptrel,-1,-1;n_standard "n_standard" true true false 2 Short 0 0,First,#,DakotaParcels,n_standard,-1,-1;ownership "ownership" true true false 30 Text 0 0,First,#,DakotaParcels,ownership,0,30;school_dst "school_dst" true true false 10 Text 0 0,First,#,DakotaParcels,school_dst,0,10;wshd_dst "wshd_dst" true true false 50 Text 0 0,First,#,DakotaParcels,wshd_dst,0,50;section "section" true true false 2 Short 0 0,First,#,DakotaParcels,section,-1,-1;township "township" true true false 2 Short 0 0,First,#,DakotaParcels,township,-1,-1;range "range" true true false 2 Short 0 0,First,#,DakotaParcels,range,-1,-1;range_dir "range_dir" true true false 2 Short 0 0,First,#,DakotaParcels,range_dir,-1,-1;prin_mer "prin_mer" true true false 2 Short 0 0,First,#,DakotaParcels,prin_mer,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,DakotaParcels,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,DakotaParcels,Shape_Area,-1,-1',
    sort_field=None
)

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\DakotaParcels_trainingnegative'>

In [2]:
# DO NOT!
# DO NOT!
# DO NOT RUN UNLESS YOU WANT TO RESET THE NEGATIVE TRAINING DATA

arcpy.conversion.ExportFeatures(
    in_features="DakotaParcels",
    out_features="DakotaParcels_trainingnegative2",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='objectid "objectid" true true false 4 Long 0 0,First,#,DakotaParcels,objectid,-1,-1;county_pin "county_pin" true true false 22 Text 0 0,First,#,DakotaParcels,county_pin,0,22;state_pin "state_pin" true true false 28 Text 0 0,First,#,DakotaParcels,state_pin,0,28;anumberpre "anumberpre" true true false 15 Text 0 0,First,#,DakotaParcels,anumberpre,0,15;anumber "anumber" true true false 4 Long 0 0,First,#,DakotaParcels,anumber,-1,-1;anumbersuf "anumbersuf" true true false 15 Text 0 0,First,#,DakotaParcels,anumbersuf,0,15;st_pre_mod "st_pre_mod" true true false 15 Text 0 0,First,#,DakotaParcels,st_pre_mod,0,15;st_pre_dir "st_pre_dir" true true false 9 Text 0 0,First,#,DakotaParcels,st_pre_dir,0,9;st_pre_typ "st_pre_typ" true true false 35 Text 0 0,First,#,DakotaParcels,st_pre_typ,0,35;st_pre_sep "st_pre_sep" true true false 20 Text 0 0,First,#,DakotaParcels,st_pre_sep,0,20;st_name "st_name" true true false 60 Text 0 0,First,#,DakotaParcels,st_name,0,60;st_pos_typ "st_pos_typ" true true false 15 Text 0 0,First,#,DakotaParcels,st_pos_typ,0,15;st_pos_dir "st_pos_dir" true true false 9 Text 0 0,First,#,DakotaParcels,st_pos_dir,0,9;st_pos_mod "st_pos_mod" true true false 15 Text 0 0,First,#,DakotaParcels,st_pos_mod,0,15;sub_type1 "sub_type1" true true false 12 Text 0 0,First,#,DakotaParcels,sub_type1,0,12;sub_id1 "sub_id1" true true false 30 Text 0 0,First,#,DakotaParcels,sub_id1,0,30;sub_type2 "sub_type2" true true false 12 Text 0 0,First,#,DakotaParcels,sub_type2,0,12;sub_id2 "sub_id2" true true false 30 Text 0 0,First,#,DakotaParcels,sub_id2,0,30;zip "zip" true true false 5 Text 0 0,First,#,DakotaParcels,zip,0,5;zip4 "zip4" true true false 4 Text 0 0,First,#,DakotaParcels,zip4,0,4;ctu_name "ctu_name" true true false 100 Text 0 0,First,#,DakotaParcels,ctu_name,0,100;ctu_id_txt "ctu_id_txt" true true false 8 Text 0 0,First,#,DakotaParcels,ctu_id_txt,0,8;postcomm "postcomm" true true false 40 Text 0 0,First,#,DakotaParcels,postcomm,0,40;co_code "co_code" true true false 5 Text 0 0,First,#,DakotaParcels,co_code,0,5;co_name "co_name" true true false 40 Text 0 0,First,#,DakotaParcels,co_name,0,40;state_code "state_code" true true false 2 Text 0 0,First,#,DakotaParcels,state_code,0,2;lot "lot" true true false 30 Text 0 0,First,#,DakotaParcels,lot,0,30;block "block" true true false 30 Text 0 0,First,#,DakotaParcels,block,0,30;plat_name "plat_name" true true false 150 Text 0 0,First,#,DakotaParcels,plat_name,0,150;owner_name "owner_name" true true false 100 Text 0 0,First,#,DakotaParcels,owner_name,0,100;owner_more "owner_more" true true false 100 Text 0 0,First,#,DakotaParcels,owner_more,0,100;own_add_l1 "own_add_l1" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l1,0,100;own_add_l2 "own_add_l2" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l2,0,100;own_add_l3 "own_add_l3" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l3,0,100;own_add_l4 "own_add_l4" true true false 100 Text 0 0,First,#,DakotaParcels,own_add_l4,0,100;tax_name "tax_name" true true false 100 Text 0 0,First,#,DakotaParcels,tax_name,0,100;tax_add_l1 "tax_add_l1" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l1,0,100;tax_add_l2 "tax_add_l2" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l2,0,100;tax_add_l3 "tax_add_l3" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l3,0,100;tax_add_l4 "tax_add_l4" true true false 100 Text 0 0,First,#,DakotaParcels,tax_add_l4,0,100;landmark "landmark" true true false 150 Text 0 0,First,#,DakotaParcels,landmark,0,150;homestead "homestead" true true false 10 Text 0 0,First,#,DakotaParcels,homestead,0,10;acres_poly "acres_poly" true true false 8 Double 0 0,First,#,DakotaParcels,acres_poly,-1,-1;acres_deed "acres_deed" true true false 8 Double 0 0,First,#,DakotaParcels,acres_deed,-1,-1;emv_land "emv_land" true true false 4 Long 0 0,First,#,DakotaParcels,emv_land,-1,-1;emv_bldg "emv_bldg" true true false 4 Long 0 0,First,#,DakotaParcels,emv_bldg,-1,-1;emv_total "emv_total" true true false 4 Long 0 0,First,#,DakotaParcels,emv_total,-1,-1;tax_year "tax_year" true true false 2 Short 0 0,First,#,DakotaParcels,tax_year,-1,-1;mkt_year "mkt_year" true true false 2 Short 0 0,First,#,DakotaParcels,mkt_year,-1,-1;tax_capac "tax_capac" true true false 4 Long 0 0,First,#,DakotaParcels,tax_capac,-1,-1;total_tax "total_tax" true true false 4 Long 0 0,First,#,DakotaParcels,total_tax,-1,-1;spec_asses "spec_asses" true true false 4 Long 0 0,First,#,DakotaParcels,spec_asses,-1,-1;useclass1 "useclass1" true true false 100 Text 0 0,First,#,DakotaParcels,useclass1,0,100;useclass2 "useclass2" true true false 100 Text 0 0,First,#,DakotaParcels,useclass2,0,100;useclass3 "useclass3" true true false 100 Text 0 0,First,#,DakotaParcels,useclass3,0,100;useclass4 "useclass4" true true false 100 Text 0 0,First,#,DakotaParcels,useclass4,0,100;multi_uses "multi_uses" true true false 10 Text 0 0,First,#,DakotaParcels,multi_uses,0,10;tax_exempt "tax_exempt" true true false 3 Text 0 0,First,#,DakotaParcels,tax_exempt,0,3;xuseclass1 "xuseclass1" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass1,0,100;xuseclass2 "xuseclass2" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass2,0,100;xuseclass3 "xuseclass3" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass3,0,100;xuseclass4 "xuseclass4" true true false 100 Text 0 0,First,#,DakotaParcels,xuseclass4,0,100;dwell_type "dwell_type" true true false 30 Text 0 0,First,#,DakotaParcels,dwell_type,0,30;home_style "home_style" true true false 30 Text 0 0,First,#,DakotaParcels,home_style,0,30;fin_sq_ft "fin_sq_ft" true true false 4 Long 0 0,First,#,DakotaParcels,fin_sq_ft,-1,-1;garage "garage" true true false 10 Text 0 0,First,#,DakotaParcels,garage,0,10;garagesqft "garagesqft" true true false 4 Long 0 0,First,#,DakotaParcels,garagesqft,-1,-1;basement "basement" true true false 10 Text 0 0,First,#,DakotaParcels,basement,0,10;heating "heating" true true false 30 Text 0 0,First,#,DakotaParcels,heating,0,30;cooling "cooling" true true false 30 Text 0 0,First,#,DakotaParcels,cooling,0,30;year_built "year_built" true true false 2 Short 0 0,First,#,DakotaParcels,year_built,-1,-1;num_units "num_units" true true false 4 Long 0 0,First,#,DakotaParcels,num_units,-1,-1;sale_date "sale_date" true true false 8 Date 0 0,First,#,DakotaParcels,sale_date,-1,-1;sale_value "sale_value" true true false 4 Long 0 0,First,#,DakotaParcels,sale_value,-1,-1;green_acre "green_acre" true true false 10 Text 0 0,First,#,DakotaParcels,green_acre,0,10;open_space "open_space" true true false 10 Text 0 0,First,#,DakotaParcels,open_space,0,10;ag_preserv "ag_preserv" true true false 10 Text 0 0,First,#,DakotaParcels,ag_preserv,0,10;agpre_enrd "agpre_enrd" true true false 8 Date 0 0,First,#,DakotaParcels,agpre_enrd,-1,-1;agpre_expd "agpre_expd" true true false 8 Date 0 0,First,#,DakotaParcels,agpre_expd,-1,-1;abb_legal "abb_legal" true true false 254 Text 0 0,First,#,DakotaParcels,abb_legal,0,254;edit_date "edit_date" true true false 8 Date 0 0,First,#,DakotaParcels,edit_date,-1,-1;exp_date "exp_date" true true false 8 Date 0 0,First,#,DakotaParcels,exp_date,-1,-1;polyptrel "polyptrel" true true false 2 Short 0 0,First,#,DakotaParcels,polyptrel,-1,-1;n_standard "n_standard" true true false 2 Short 0 0,First,#,DakotaParcels,n_standard,-1,-1;ownership "ownership" true true false 30 Text 0 0,First,#,DakotaParcels,ownership,0,30;school_dst "school_dst" true true false 10 Text 0 0,First,#,DakotaParcels,school_dst,0,10;wshd_dst "wshd_dst" true true false 50 Text 0 0,First,#,DakotaParcels,wshd_dst,0,50;section "section" true true false 2 Short 0 0,First,#,DakotaParcels,section,-1,-1;township "township" true true false 2 Short 0 0,First,#,DakotaParcels,township,-1,-1;range "range" true true false 2 Short 0 0,First,#,DakotaParcels,range,-1,-1;range_dir "range_dir" true true false 2 Short 0 0,First,#,DakotaParcels,range_dir,-1,-1;prin_mer "prin_mer" true true false 2 Short 0 0,First,#,DakotaParcels,prin_mer,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,DakotaParcels,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,DakotaParcels,Shape_Area,-1,-1',
    sort_field=None
)

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\DakotaParcels_trainingnegative2'>

In [11]:
import numpy as np

#  A) LOAD THE PARCEL IDs OF THE TRAINING DATA 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
analysisFields = ['acres_poly', 'useclass2', 'distance_fws_NN',
       'distance_dnr_NN', 'distance_prairie_NN', 'distance_all_NN',
       'usd_per_acre_total', 'usd_per_acre_bldg', 'usd_per_acre_land']
# TODO: ADD <$500/Acre cutoff!


# Pass dataframe by value
dfPP = dfProspectiveParcels.copy()
dfMLParcels = dfPP.copy()
dfMLParcels = dfMLParcels[analysisFields]
mLParcels = dfMLParcels.to_numpy().astype(float)

# READ
all_fields = ['objectid']
fc1 = 'DakotaParcels_trainingnegative'
fc2 = 'DakotaParcels_trainingpositive'


# LOAD PARCEL DATA TO NP ARRAY
negative_parcelOID = arcpy.da.FeatureClassToNumPyArray(fc1, all_fields, "")
positive_parcelOID = arcpy.da.FeatureClassToNumPyArray(fc2, all_fields, "")

x_n = []
x_p = []

# FIND TRAINING PARCELS
# --------------------------
prosOID = dfPP['objectid'].to_numpy()
numParcels = prosOID.shape[0]

for item in positive_parcelOID['objectid']:
    x = np.where(prosOID == item)
    if len(x)>0:
        if len(x[0])>0:
            x_p.append(x[0][0])

for item in negative_parcelOID['objectid']:
    x = np.where(prosOID == item)
    if len(x)>0:
        if len(x[0])>0:
            x_n.append(x[0][0])
#--------------------------
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#   B) CREATE MASKs ASSOCIATING THE TRAINING DATA TO PROSPECTIVE DATA
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
nIndexArray = np.array(x_n, dtype=int)
pIndexArray = np.array(x_p, dtype=int)

nMaskArray = np.zeros(prosOID.shape[0], dtype=int)
nMaskArray[nIndexArray] = 1

pMaskArray = np.zeros(prosOID.shape[0], dtype=int)
pMaskArray[pIndexArray] = 1

uMaskArray = np.logical_not((nMaskArray + pMaskArray).astype(bool))
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#   C) SANITIZE THE PARCEL DATA FOR NANs or INFs
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#random_choice_size = 100000
#ZIndexArray = np.random.choice(uIndexArray, random_choice_size)
Z = mLParcels#[ZIndexArray]
ZIndexArray = numpy.arange(0, Z.shape[0], 1, dtype=int)

Znan = np.argwhere(np.logical_or(np.isnan(Z).any(axis=1),np.isinf(Z).any(axis=1))).flatten()

Z = numpy.delete(Z, Znan, axis = 0)

# Input new index get old index (ZIndexArray)
ZIndexArray = numpy.delete(ZIndexArray, Znan, axis = 0)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#   D) CONVERT TRAINING DAtA INTO A ML FORMAT
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X = np.vstack((mLParcels[nIndexArray], mLParcels[pIndexArray]))


Y = np.zeros((len(nIndexArray)+len(pIndexArray)))
Y[0:len(nIndexArray)] = 0
Y[len(nIndexArray):(len(nIndexArray)+len(pIndexArray))] = 1
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [1]:
# SHOW PCA ANALYSIS OF DATA
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaling=StandardScaler()
 
# Use fit and transform method 
scaling.fit(Z)
Scaled_fit_data=scaling.transform(Z)
Scaled_predict_data=scaling.transform(X)
 
# Set the n_components=2
principal=PCA(n_components=6)
principal.fit(Scaled_fit_data)
x=principal.transform(Scaled_predict_data)

plt.scatter(x[:,0],x[:,1], c=Y,cmap='plasma', alpha=0.3)
plt.xlabel('pc1')
plt.ylabel('pc2')
plt.show()

#plt.scatter(x[:,0],x[:,2], c=Y,cmap='plasma', alpha=0.3)
#plt.xlabel('pc1')
#plt.ylabel('pc3')
#plt.show()

#plt.scatter(x[:,0],x[:,3], c=Y,cmap='plasma', alpha=0.3)
#plt.xlabel('pc1')
#plt.ylabel('pc4')
#plt.show()

#plt.scatter(x[:,0],x[:,4], c=Y,cmap='plasma', alpha=0.3)
#plt.xlabel('pc1')
#plt.ylabel('pc5')
#plt.show()

print("Fraction of explained variance: " + str(sum(principal.explained_variance_ratio_)))
plt.show()

NameError: name 'Z' is not defined

In [13]:
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#   A) NORMALIZE THE TRAINING AND TESTING DATA
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
scaling=StandardScaler()
 
# Use fit and transform method 
scaling.fit(Z)
Scaled_X_data=scaling.transform(X)
Scaled_Z_data=scaling.transform(Z)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#   B) SIMPLIFY DATA USING PRINCIPAL COMPONENT ANALYSIS
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set the n_components s= 2
principal = PCA(n_components = 4)

principal.fit(Scaled_Z_data)

reduced_Z_data = principal.transform(Scaled_Z_data)
reduced_X_data = principal.transform(Scaled_X_data)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#   C) APPLY THE SEPERATION USING A SUPPORT VECTOR MACHINE WITH AN RBF KERNEL
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# SVM RADIAL KERNEL
clf = svm.SVC(probability=True)
clf.fit(reduced_X_data, Y)
result = clf.predict_proba(reduced_Z_data)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#   D) SELECT THE TOP 25 PICKS ON THE MAP
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# RANKS THE TOP PICKS
rankedObjectID = dfPP.loc[ZIndexArray[numpy.argsort(result[:,1])[::-1]]]#['objectid'].tolist()
rankedObjectID['confidence'] = numpy.sort(result[:,1])[::-1]
#top25 = ZIndexArray[result[:,1]>0.99]
#top25 = dfPP.iloc[ZIndexArray[numpy.argsort(result[:,0])]]['objectid'].tolist()[:25]

# SELECT THE TOP 25 PICK
import os
o_fields = ['OID@', 'objectid', 'county_pin', 'state_pin', 'own_add_l1', 'tax_name', 'acres_poly', 'emv_land', 'emv_bldg', 'emv_total', 'useclass1', 'useclass2','SHAPE@TRUECENTROID','confidence']
fc = 'DakotaParcels'
local_path = arcpy.Describe(fc).path
parcel_numpy = arcpy.da.FeatureClassToNumPyArray(fc, o_fields, "")
spatial_ref = arcpy.Describe(fc).spatialReference
#parcel_numpy['confidence'][ZIndexArray] = result[:,1]
#print(parcel_numpy['confidence'])

# Create the update cursor
cursor = arcpy.UpdateCursor('DakotaParcels')

with arcpy.da.UpdateCursor('DakotaParcels', ["objectid", "confidence"]) as cursor:
    for row in cursor:
        r = rankedObjectID.loc[rankedObjectID['objectid'] == row[0]]
        if len(r) != 0:
            row[1] = float(r['confidence'])
            cursor.updateRow(row)

In [14]:
# SET THE DISTANCE TO LAND COLUMN
cursor = arcpy.UpdateCursor('DakotaParcels')

with arcpy.da.UpdateCursor('DakotaParcels', ["objectid", "distancetoland"]) as cursor:
    for row in cursor:
        r = dfProspectiveParcels.loc[dfProspectiveParcels['objectid'] == row[0]]
        if len(r) != 0:
            row[1] = float(r['distance_all_NN'])
            cursor.updateRow(row)

In [13]:
# SELECT TOP 100!
for item in rankedObjectID['objectid'].tolist()[:100]:
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="DakotaParcels",
        selection_type="ADD_TO_SELECTION",
        where_clause="objectid = "+ str(item),
        invert_where_clause = None
    )
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# CALCULATE True Positive (good good picks) and False Positive (bad good picks) rate!

In [16]:
# SELECT BOTTOM 100!
for item in rankedObjectID['objectid'].tolist()[-100:]:
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view="DakotaParcels",
        selection_type="ADD_TO_SELECTION",
        where_clause="objectid = "+ str(item),
        invert_where_clause = None
    )
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# CALCULATE True Negative (good bad picks) and False Negative (bad bad picks) rate!

In [2]:
# convert parcel land type to raster

arcpy.conversion.PolygonToRaster("DakotaParcels", "useclass1", "ParcelLanduseRaster", "MAXIMUM_COMBINED_AREA")

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\ParcelLanduseRaster'>

In [4]:
# reclassify land type to binary preferred/nonpreferred land types

arcpy.ddd.Reclassify(
    in_raster="ParcelLanduseRaster",
    reclass_field="useclass1",
    remap="Residential 1;Residential-Townhouse 1;Apartment 1;'Ag-Ag Preserve' 2;Exempt 2;'Ag-Green Acres' 2;Commercial-Preferred 1;Ag 2;Industrial-Preferred 1;Commercial 1;Machinery 1;'Commercial-Open Space' 2;Industrial 1;Utilities 1;'Apartment-Mobile Home Park' 1;Railroad 1;'Apartment-Mobile Home' 1;'Commercial-Preferred-Open Space' 1;Utilities-Preferred 1;'Apartment-Nursing Home' 20",
    out_raster=r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\Reclass_Parc1",
    missing_values="DATA"
)

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\Reclass_Parc1'>

In [2]:
# convert parcel cost per acre to raster

arcpy.conversion.PolygonToRaster("DakotaParcels", "costperacre", "ParcelCostRaster", "MAXIMUM_COMBINED_AREA")

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\ParcelCostRaster'>

In [5]:
# reclassify into integer categories

arcpy.ddd.Reclassify(
    in_raster="ParcelCostRaster",
    reclass_field="Value",
    remap="4 5375 1;5375 9914 2;9914 14039 3;14039 18891 4;18891 24223 5;24223 29000 6;29000 33026 7;33026 37083 8;37083 41983 9;41983 46026 10",
    out_raster=r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\Reclass_Parc2",
    missing_values="DATA"
)

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\Reclass_Parc2'>

In [20]:
# convert parcel land distance to raster

arcpy.conversion.PolygonToRaster("DakotaParcels", "distancetoland", "ParcelDistanceRaster", "MAXIMUM_COMBINED_AREA")

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\ParcelDistanceRaster'>

In [7]:
# reclassify to integer values

arcpy.ddd.Reclassify(
    in_raster="ParcelDistanceRaster",
    reclass_field="VALUE",
    remap="27.075730469 865.102307524 1;865.102307524 1563.457788403 2;1563.457788403 2226.895495238 3;2226.895495238 2890.333202073 4;2890.333202073 3588.688682952 5;3588.688682952 4287.044163831 6;4287.044163831 4985.399644710 7;4985.399644710 5753.590673676 8;5753.590673676 6766.206120951 9;6766.206120951 8931.108111675 10",
    out_raster=r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\Reclass_Parc3",
    missing_values="DATA"
)

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\finalproject\\finalproject.gdb\\Reclass_Parc3'>

In [8]:
# create weighted raster

out_raster = arcpy.sa.WeightedOverlay(
    in_weighted_overlay_table=r"('C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\Reclass_Parc1' 34 'useclass1' ('Residential/Residential-Townhouse/Apartment/Commercial-Preferred/Industrial-Preferred/Commercial/Machinery/Industrial/Utilities/' 1; 'Ag-Ag Preserve/Exempt/Ag-Green Acres/Ag/Commercial-Open Space' 9; 'Apartment-Nursing Home' 1; NODATA NODATA); 'C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\Reclass_Parc2' 33 'Value' (1 9; 2 8; 3 7; 4 6; 5 5; 6 4; 7 3; 8 2; 9 1; 10 1; NODATA NODATA); 'C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\Reclass_Parc3' 33 'Value' (1 9; 2 8; 3 7; 4 6; 5 5; 6 4; 7 3; 8 2; 9 1; 10 1; NODATA NODATA));1 9 1"
)
out_raster.save(r"C:\Users\18284\Documents\ArcGIS\Projects\finalproject\finalproject.gdb\WeightedRaster")